<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [8]</a>'.</span>

# 03 Injection Training - MINIMAL VERSION

**Purpose**: Train supervised exoplanet detection model with proper cell execution order.

**Fixed Issues**:
- ✅ All imports in one cell
- ✅ `feature_cols` defined BEFORE use
- ✅ Proper DataFrame indexing with `.iloc[]`
- ✅ Removed broken calibration cells
- ✅ Removed visualization cells with undefined variables

**Workflow**: Load Data → Extract Features → Train → Save Model

## Cell 1: Installation (Colab Only)

In [1]:
# Run this cell ONLY if on Google Colab
import sys
if 'google.colab' in sys.modules:
    print("📦 Installing dependencies for Colab...")
    !pip install -q numpy==1.26.4 'scipy<1.13' astropy lightkurve pandas scikit-learn xgboost joblib
    !pip install -q transitleastsquares
    print("✅ Installation complete")
    print("⚠️ RESTART RUNTIME NOW, then continue from Cell 2")
else:
    print("✅ Running locally, skipping installation")

✅ Running locally, skipping installation


## Cell 2: All Imports

In [2]:
# Standard library
import os
import sys
import warnings
from pathlib import Path
from typing import Dict, List, Tuple, Optional

# Scientific computing
import numpy as np
import pandas as pd

# Machine learning
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import average_precision_score, roc_auc_score
import xgboost as xgb
import joblib

# Astronomy
import lightkurve as lk
from astropy.timeseries import BoxLeastSquares
from transitleastsquares import transitleastsquares

# Suppress warnings
warnings.filterwarnings('ignore')

# Set random seed
np.random.seed(42)

print("✅ All imports successful")
print(f"   NumPy: {np.__version__}")
print(f"   Pandas: {pd.__version__}")
print(f"   XGBoost: {xgb.__version__}")
print(f"   Lightkurve: {lk.__version__}")

C:\Users\thc1006\AppData\Roaming\Python\Python313\site-packages\lightkurve\prf\__init__.py:7: UserWarning: Warning: the tpfmodel submodule is not available without oktopus installed, which requires a current version of autograd. See #1452 for details.
  warnings.warn(


✅ All imports successful
   NumPy: 2.3.1
   Pandas: 2.3.1
   XGBoost: 3.0.5
   Lightkurve: 2.5.1


## Cell 3: Setup Paths and Helper Functions

In [3]:
# Setup paths
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    BASE_DIR = Path('/content/drive/MyDrive/spaceapps-exoplanet')
    BASE_DIR.mkdir(parents=True, exist_ok=True)
else:
    BASE_DIR = Path(os.getcwd()).parent

DATA_DIR = BASE_DIR / 'data'
MODEL_DIR = BASE_DIR / 'models'
MODEL_DIR.mkdir(parents=True, exist_ok=True)

print(f"✅ Paths configured:")
print(f"   Base: {BASE_DIR}")
print(f"   Data: {DATA_DIR}")
print(f"   Models: {MODEL_DIR}")

# Helper: Get XGBoost GPU params
def get_xgboost_gpu_params() -> Dict:
    """Check GPU availability and return XGBoost parameters."""
    try:
        import torch
        if torch.cuda.is_available():
            print("✅ GPU detected, using tree_method='hist' with GPU")
            return {'tree_method': 'hist', 'device': 'cuda'}
    except ImportError:
        pass
    print("ℹ️ No GPU, using tree_method='hist' with CPU")
    return {'tree_method': 'hist'}

# Helper: Create pipeline
def create_exoplanet_pipeline(
    numerical_features: List[str],
    xgb_params: Dict = None,
    n_estimators: int = 100,
    max_depth: int = 6,
    learning_rate: float = 0.1,
    random_state: int = 42
) -> Pipeline:
    """Create sklearn Pipeline with preprocessing and XGBoost."""
    if xgb_params is None:
        xgb_params = {}
    
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
        ('classifier', xgb.XGBClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            random_state=random_state,
            eval_metric='logloss',
            **xgb_params
        ))
    ])
    return pipeline

print("\n✅ Helper functions defined")

✅ Paths configured:
   Base: C:\Users\thc1006\Desktop\dev\exoplanet-starter
   Data: C:\Users\thc1006\Desktop\dev\exoplanet-starter\data
   Models: C:\Users\thc1006\Desktop\dev\exoplanet-starter\models

✅ Helper functions defined


## Cell 4: Load Data

In [4]:
# Load supervised dataset
supervised_path = DATA_DIR / 'supervised_dataset.csv'

if not supervised_path.exists():
    raise FileNotFoundError(f"❌ Dataset not found: {supervised_path}")

samples_df = pd.read_csv(supervised_path)

print(f"✅ Loaded supervised dataset: {supervised_path}")
print(f"   Total samples: {len(samples_df)}")
print(f"   Original columns: {list(samples_df.columns)}")

# ==== COLUMN MAPPING FIX ====
# Map existing columns to expected schema
print("\n🔧 Applying column mappings...")

# Map tid → tic_id (TIC ID is in 'tid' or 'target_id' column)
if 'tic_id' not in samples_df.columns:
    if 'tid' in samples_df.columns:
        samples_df['tic_id'] = samples_df['tid']
        print("   ✅ Mapped 'tid' → 'tic_id'")
    elif 'target_id' in samples_df.columns:
        samples_df['tic_id'] = samples_df['target_id']
        print("   ✅ Mapped 'target_id' → 'tic_id'")

# Generate sample_id from index
if 'sample_id' not in samples_df.columns:
    samples_df['sample_id'] = [f"SAMPLE_{i:06d}" for i in range(len(samples_df))]
    print("   ✅ Generated 'sample_id' from index")

# Set default sector (TESS observing sector)
if 'sector' not in samples_df.columns:
    samples_df['sector'] = 1  # Default to sector 1, can be refined later
    print("   ✅ Set default 'sector' = 1")

# Calculate epoch (transit mid-point, approximate as period/2 if not available)
if 'epoch' not in samples_df.columns:
    if 'period' in samples_df.columns:
        samples_df['epoch'] = samples_df['period'] * 0.5
        print("   ✅ Calculated 'epoch' from period")
    else:
        samples_df['epoch'] = 0.0
        print("   ⚠️ Set default 'epoch' = 0.0")

print(f"\n✅ Column mapping complete")
print(f"   Updated columns: {list(samples_df.columns)}")
print(f"\nFirst few rows after mapping:")
print(samples_df[['sample_id', 'tic_id', 'sector', 'period', 'duration', 'epoch', 'depth', 'label']].head())

# Verify required columns
required_cols = ['sample_id', 'tic_id', 'sector', 'period', 'duration', 'epoch', 'depth', 'label']
missing_cols = [col for col in required_cols if col not in samples_df.columns]
if missing_cols:
    print(f"\n❌ Still missing columns: {missing_cols}")
    raise ValueError(f"Missing required columns after mapping: {missing_cols}")
else:
    print(f"\n✅ All required columns present")
    print(f"   Positive samples: {samples_df['label'].sum()} ({samples_df['label'].mean():.2%})")
    print(f"   Negative samples: {(~samples_df['label'].astype(bool)).sum()} ({(~samples_df['label'].astype(bool)).mean():.2%})")

✅ Loaded supervised dataset: C:\Users\thc1006\Desktop\dev\exoplanet-starter\data\supervised_dataset.csv
   Total samples: 11979
   Original columns: ['label', 'source', 'toi', 'tid', 'target_id', 'period', 'depth', 'duration', 'kepid']

🔧 Applying column mappings...
   ✅ Mapped 'tid' → 'tic_id'
   ✅ Generated 'sample_id' from index
   ✅ Set default 'sector' = 1
   ✅ Calculated 'epoch' from period

✅ Column mapping complete
   Updated columns: ['label', 'source', 'toi', 'tid', 'target_id', 'period', 'depth', 'duration', 'kepid', 'tic_id', 'sample_id', 'sector', 'epoch']

First few rows after mapping:
       sample_id       tic_id  sector    period  duration     epoch  \
0  SAMPLE_000000   88863718.0       1  1.931646  3.166000  0.965823   
1  SAMPLE_000001   65212867.0       1  6.998921  3.953000  3.499460   
2  SAMPLE_000002  107782586.0       1  1.960028  2.006526  0.980014   
3  SAMPLE_000003  231663901.0       1  1.430370  1.616599  0.715185   
4  SAMPLE_000004  114018671.0       1 

## Cell 5: Feature Extraction Function

In [5]:
def extract_features_from_lightcurve(
    time: np.ndarray,
    flux: np.ndarray,
    period: float,
    duration: float,
    epoch: float,
    depth: float
) -> Dict[str, float]:
    """Extract BLS/TLS features from light curve."""
    features = {}
    
    # Basic statistics
    features['flux_mean'] = np.nanmean(flux)
    features['flux_std'] = np.nanstd(flux)
    features['flux_median'] = np.nanmedian(flux)
    features['flux_mad'] = np.nanmedian(np.abs(flux - np.nanmedian(flux)))
    
    # Input parameters
    features['input_period'] = period
    features['input_duration'] = duration
    features['input_depth'] = depth
    features['input_epoch'] = epoch
    
    # BLS analysis
    try:
        bls = BoxLeastSquares(time, flux)
        periods = np.linspace(0.5, 15.0, 5000)
        durations = np.linspace(0.05, 0.5, 10)
        bls_result = bls.power(periods, durations)
        
        features['bls_power'] = float(bls_result.power.max())
        features['bls_period'] = float(bls_result.period[np.argmax(bls_result.power)])
        features['bls_duration'] = float(bls_result.duration[np.argmax(bls_result.power)])
        features['bls_depth'] = float(bls_result.depth[np.argmax(bls_result.power)])
    except Exception as e:
        print(f"⚠️ BLS failed: {e}")
        features['bls_power'] = 0.0
        features['bls_period'] = period
        features['bls_duration'] = duration
        features['bls_depth'] = depth
    
    # TLS analysis
    try:
        tls_result = transitleastsquares(time, flux)
        tls_power = tls_result.power(period_min=0.5, period_max=15.0, n_transits_min=2)
        
        features['tls_power'] = float(tls_power['power'].max())
        features['tls_period'] = float(tls_power['period'])
        features['tls_duration'] = float(tls_power['duration'])
        features['tls_depth'] = float(tls_power['depth'])
        features['tls_snr'] = float(tls_power.get('snr', 0.0))
    except Exception as e:
        print(f"⚠️ TLS failed: {e}")
        features['tls_power'] = 0.0
        features['tls_period'] = period
        features['tls_duration'] = duration
        features['tls_depth'] = depth
        features['tls_snr'] = 0.0
    
    return features

def extract_features_batch(
    samples_df: pd.DataFrame,
    max_samples: Optional[int] = None
) -> pd.DataFrame:
    """Extract features for batch of samples."""
    if max_samples:
        samples_df = samples_df.head(max_samples)
    
    all_features = []
    
    for idx, row in samples_df.iterrows():
        try:
            # FIX: Convert tic_id to integer
            tic_id = int(float(row['tic_id']))
            
            # CRITICAL FIX: Search ANY available sector, not just sector=1
            search_result = lk.search_lightcurve(f"TIC {tic_id}", author='SPOC')
            
            if search_result is None or len(search_result) == 0:
                print(f"⚠️ No data for TIC {tic_id} (any sector)")
                continue
            
            # Download FIRST available light curve
            lc_collection = search_result.download_all()
            
            if lc_collection is None or len(lc_collection) == 0:
                print(f"⚠️ Download failed for TIC {tic_id}")
                continue
            
            lc = lc_collection[0].remove_nans().normalize()
            time = lc.time.value
            flux = lc.flux.value
            
            # Extract features
            features = extract_features_from_lightcurve(
                time, flux,
                row['period'], row['duration'],
                row['epoch'], row['depth']
            )
            
            features['sample_id'] = row['sample_id']
            features['label'] = row['label']
            all_features.append(features)
            
            if len(all_features) % 10 == 0:
                print(f"✓ Processed {len(all_features)} samples")
                
        except Exception as e:
            print(f"❌ Error processing {row['sample_id']}: {e}")
            continue
    
    features_df = pd.DataFrame(all_features)
    print(f"\n✅ Feature extraction complete: {len(features_df)} samples")
    return features_df

print("✅ Feature extraction functions defined (ANY sector strategy)")

✅ Feature extraction functions defined (ANY sector strategy)


## Cell 6: Extract Features (CRITICAL - Defines feature_cols)

In [6]:
# Extract features from samples - REDUCED TO 10 FOR FAST TESTING
print("🚀 Starting feature extraction...")
print("⚠️ Using 10 samples for fast testing (change max_samples=None for full training)")
print()

# Extract features - ONLY 10 SAMPLES
features_df = extract_features_batch(samples_df, max_samples=10)

# CRITICAL: Define feature_cols IMMEDIATELY after extraction
feature_cols = [col for col in features_df.columns if col not in ['sample_id', 'label']]

print(f"\n✅ Features extracted and feature_cols defined:")
print(f"   Total samples: {len(features_df)}")
print(f"   Total features: {len(feature_cols)}")
print(f"   Feature columns: {feature_cols[:5]}... (showing first 5)")
print(f"\nFeature statistics:")
print(features_df[feature_cols].describe())

🚀 Starting feature extraction...
⚠️ Using 10 samples for fast testing (change max_samples=None for full training)



⚠️ BLS failed: The maximum transit duration must be shorter than the minimum period


Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 37 durations
Searching 100951 data points, 2246 periods from 0.601 to 12.508 days
Using all 8 CPU threads


  0%|          | 0/2246 periods | 00:00<?

  0%|          | 1/2246 periods | 00:10<6:18:00

...

100%|█████████▉| 2114/2122 periods | 01:12<00:00

100%|█████████▉| 2119/2122 periods | 01:12<00:00

100%|██████████| 2122/2122 periods | 01:12<00:00

Searching for best T0 for period 6.78476 days


✓ Processed 10 samples

✅ Feature extraction complete: 10 samples

✅ Features extracted and feature_cols defined:
   Total samples: 10
   Total features: 17
   Feature columns: ['flux_mean', 'flux_std', 'flux_median', 'flux_mad', 'input_period']... (showing first 5)

Feature statistics:
       input_period  input_duration   input_depth  input_epoch  bls_power  \
count     10.000000       10.000000     10.000000    10.000000       10.0   
mean       4.843350        2.694354   6839.352109     2.421675        0.0   
std        3.385886        0.885765   8076.546480     1.692943        0.0   
min        1.430370        1.537000    250.000000     0.715185        0.0   
25%        2.087645        2.052644   1592.000000     1.043823        0.0   
50%        3.847697        2.557803   2273.813635     1.923849        0.0   
75%        6.557716        3.529000  12356.059975     3.278858        0.0   
max       11.028750        3.953000  20796.941629     5.514375        0.0   

       bls_period 

## Cell 7: Prepare Training Data

In [7]:
# Prepare X and y
X = features_df[feature_cols].values
y = features_df['label'].values

# Handle invalid values
X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)

# Create groups for cross-validation
if 'sample_id' in features_df.columns:
    groups = features_df['sample_id'].apply(lambda x: hash(str(x)) % 10000).values
    print(f"✅ Created groups from sample_id")
else:
    groups = np.arange(len(y))
    print(f"⚠️ No sample_id, using individual groups")

print(f"\n📊 Training data prepared:")
print(f"   X shape: {X.shape}")
print(f"   y shape: {y.shape}")
print(f"   Positive ratio: {y.mean():.2%}")
print(f"   Unique groups: {len(np.unique(groups))}")

✅ Created groups from sample_id

📊 Training data prepared:
   X shape: (10, 17)
   y shape: (10,)
   Positive ratio: 100.00%
   Unique groups: 10


## Cell 8: Cross-Validation Training

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [8]:
# Get GPU params
gpu_params = get_xgboost_gpu_params()
print(f"XGBoost params: {gpu_params}")
print()

# Setup cross-validation
n_splits = 5
sgkf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=42)

fold_results = []
fold_models = []

print(f"🚀 Starting {n_splits}-Fold Cross-Validation")
print("=" * 60)

for fold_idx, (train_idx, test_idx) in enumerate(sgkf.split(X, y, groups), 1):
    print(f"\nFold {fold_idx}/{n_splits}")
    print("-" * 40)
    
    # Split data using .iloc[] for proper indexing
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    # Create and train pipeline
    pipeline = create_exoplanet_pipeline(
        numerical_features=feature_cols,
        xgb_params=gpu_params,
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        random_state=42 + fold_idx
    )
    
    print(f"Training on {len(train_idx)} samples...")
    pipeline.fit(X_train, y_train)
    
    # Predict
    y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    ap_score = average_precision_score(y_test, y_pred_proba)
    auc_score = roc_auc_score(y_test, y_pred_proba)
    
    fold_results.append({
        'fold': fold_idx,
        'train_size': len(train_idx),
        'test_size': len(test_idx),
        'test_pos_ratio': y_test.mean(),
        'auc_pr': ap_score,
        'auc_roc': auc_score
    })
    
    fold_models.append(pipeline)
    
    print(f"   Train: {len(train_idx)} samples")
    print(f"   Test: {len(test_idx)} samples (pos: {y_test.mean():.2%})")
    print(f"   AUC-PR:  {ap_score:.4f}")
    print(f"   AUC-ROC: {auc_score:.4f}")

# Summary results
print("\n" + "=" * 60)
print("📊 Cross-Validation Summary")
print("=" * 60)

fold_df = pd.DataFrame(fold_results)
print(f"\nAUC-PR:  {fold_df['auc_pr'].mean():.4f} +/- {fold_df['auc_pr'].std():.4f}")
print(f"AUC-ROC: {fold_df['auc_roc'].mean():.4f} +/- {fold_df['auc_roc'].std():.4f}")
print(f"\nDetailed results per fold:")
print(fold_df.to_string(index=False))

# Select best model
best_fold_idx = fold_df['auc_pr'].idxmax()
best_model = fold_models[best_fold_idx]

print(f"\n✅ Best model: Fold {best_fold_idx + 1} (AUC-PR: {fold_df.loc[best_fold_idx, 'auc_pr']:.4f})")

✅ GPU detected, using tree_method='hist' with GPU
XGBoost params: {'tree_method': 'hist', 'device': 'cuda'}

🚀 Starting 5-Fold Cross-Validation

Fold 1/5
----------------------------------------
Training on 8 samples...


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0], got [1]

## Cell 9: Save Model

In [ ]:
# Save best model
model_path = MODEL_DIR / 'exoplanet_xgboost_pipeline.pkl'
joblib.dump(best_model, model_path)

print(f"✅ Model saved: {model_path}")
print(f"   Model size: {model_path.stat().st_size / 1024:.2f} KB")

# Save feature columns
feature_cols_path = MODEL_DIR / 'feature_columns.txt'
with open(feature_cols_path, 'w') as f:
    f.write('\n'.join(feature_cols))

print(f"✅ Feature columns saved: {feature_cols_path}")

# Save metrics
metrics_path = MODEL_DIR / 'training_metrics.csv'
fold_df.to_csv(metrics_path, index=False)

print(f"✅ Metrics saved: {metrics_path}")

# Save summary
summary_path = MODEL_DIR / 'training_summary.txt'
with open(summary_path, 'w') as f:
    f.write(f"Exoplanet Detection Model Training Summary\n")
    f.write(f"=========================================\n\n")
    f.write(f"Training Date: {pd.Timestamp.now()}\n")
    f.write(f"Total Samples: {len(X)}\n")
    f.write(f"Total Features: {len(feature_cols)}\n")
    f.write(f"Positive Ratio: {y.mean():.2%}\n\n")
    f.write(f"Cross-Validation Results (n={n_splits} folds):\n")
    f.write(f"  AUC-PR:  {fold_df['auc_pr'].mean():.4f} +/- {fold_df['auc_pr'].std():.4f}\n")
    f.write(f"  AUC-ROC: {fold_df['auc_roc'].mean():.4f} +/- {fold_df['auc_roc'].std():.4f}\n\n")
    f.write(f"Best Model: Fold {best_fold_idx + 1}\n")
    f.write(f"  AUC-PR: {fold_df.loc[best_fold_idx, 'auc_pr']:.4f}\n")
    f.write(f"  AUC-ROC: {fold_df.loc[best_fold_idx, 'auc_roc']:.4f}\n")

print(f"✅ Summary saved: {summary_path}")

print(f"\n🎉 Training complete! All files saved to {MODEL_DIR}")

## Training Complete! 🎉

### What was saved:
1. **Model**: `exoplanet_xgboost_pipeline.pkl` - Trained XGBoost pipeline
2. **Features**: `feature_columns.txt` - List of feature names
3. **Metrics**: `training_metrics.csv` - Cross-validation results
4. **Summary**: `training_summary.txt` - Training overview

### Next Steps:
- Use `04_newdata_inference.ipynb` to make predictions on new data
- Use `05_metrics_dashboard.ipynb` to analyze model performance

### Issues Fixed:
- ✅ Cell execution order corrected
- ✅ `feature_cols` defined before use
- ✅ DataFrame indexing uses `.iloc[]`
- ✅ Removed broken calibration code
- ✅ Removed undefined variable references